# CatBoost Tryout

Testing and integration of CatBoost in MERCS. Catboost can handle missing values and therefore is rather interesting to us.

In [36]:
import catboost as cb

from catboost.datasets import titanic

from mercs.tests import load_iris
import numpy as np

In [37]:
iris = load_iris()

train = iris[0]
test = iris[1]
nominal_ids = iris[2]

In [38]:
train_df, test_df = titanic()
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

X = train_df.drop('Survived', axis=1)
y = train_df.Survived

nominal_ids = np.where(X.dtypes != np.float)[0]

from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

X_test = test_df

train_pool = Pool(X_train, y_train, cat_features=nominal_ids)
validate_pool = Pool(X_validation, y_validation, cat_features=nominal_ids )

In [39]:
types = list(train_df.dtypes)
types

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('O')]

In [33]:
train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [27]:
import pandas as pd

In [35]:
pd.api.types.is_object_dtype(types[4])

True

In [4]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

In [58]:
model = CatBoostClassifier(
    custom_loss=["Accuracy"], random_seed=42, logging_level="Silent"
)

In [56]:
params = dict(
    custom_loss=["Accuracy"],
    iterations=100,
    learning_rate=0.1,
    logging_level="Silent",
    od_type= 'Iter',
    od_wait= 40
)

mod = CatBoostClassifier(**params)

In [57]:
%%timeit
mod.fit(X_train, y_train,
    cat_features=nominal_ids,)

159 ms ± 55.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [75]:
desc_ids = {1,2,3,4}

nominal_ids = {1,4,2}

In [68]:
np.array(list(desc_ids))

array([1, 2, 3, 4])

In [76]:
np.where(np.in1d(np.array(list(desc_ids)), np.array(list(nominal_ids))))[0]

array([0, 1, 3])

In [ ]:
def get_cat_features(desc_ids, nominal_ids):
    return np.where(np.in1d(np.array(list(desc_ids)), np.array(list(nominal_ids))))[0]